In [ ]:
# BASIC IMPORTS
import sys
sys.path.append(r'..\src')

import pandas as pd

import pylab as plt

import seaborn as sns

import numpy as np

import re

from src import getGraphic
from ufc_fights import set_names_upper, set_name_winner, fix_columns, set_nan_columns, add_fighterid
from ufc_fighters import load_ufc, set_fighter_upper, clean_fighters

In [ ]:
# MACHINE LEARNING IMPORTS
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor as RFR
rfr=RFR()

from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2

import h2o
from h2o.automl import H2OAutoML

In [ ]:
fighter = pd.read_csv(r'..\data\raw_fighter_details.csv')

data = pd.read_csv(r'..\data\data.csv')

ufc_fighters = pd.read_csv(r'..\data\ufc_fighters_official.csv')

ufc_odds = pd.read_csv('../data/ufc_odds.csv')

# TABLA LUCHADORES

In [ ]:
'''set_fighter_upper(fighter)

fighters = clean_fighters(fighter, ufc_fighters)

fighters'''

# TABLA FIGHTS

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
# Las reglas oficiales de la UFC no entraron hasta el año 2001, así que solo cogemos combates por encima del año 2002
data = data[data.date>'2002-01-01']
# B_draw y R_draw son columnas con todos los valores 0
data = data.drop(['B_draw','R_draw'], axis=1)
# Eliminamos DRAW en Winner para ir probando
data = data[data.Winner!='Draw']
# Eliminamos la columna LOCATION
data = data.drop('location', axis=1)

In [ ]:
set_names_upper(data)

data.head()

In [ ]:
'''set_name_winner(data)

data.head()'''

In [ ]:
fix_columns(data)

data

In [ ]:
set_nan_columns(data)

data

In [ ]:
data = data.drop('fight_id', axis=1)

In [ ]:
# ¿Borrar filas donde weight_class es CATCHWEIGHT?

In [ ]:
# AÑADIR ÚLTIMOS COMBATES PROCHAZKA Y TEIXEIRA

jp = {'R_fighter':'DOMINICK REYES', 'B_fighter':'JIRI PROCHAZKA', 'date':'2021-05-01', 'Winner':'Blue', 'title_bout':False,
      'weight_class':'LightHeavyweight', 'B_avg_KD':, 'B_avg_SIG_STR_pct', 'B_avg_TD_pct',
       'B_avg_SUB_ATT', 'B_avg_REV', 'B_avg_TD_att', 'B_avg_TD_landed',
       'B_avg_CTRL_time', 'B_total_time_fought', 'B_total_rounds_fought',
       'B_total_title_bouts', 'B_wins', 'B_losses',
       'B_win_by_Decision_Majority', 'B_win_by_Decision_Split',
       'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission',
       'B_win_by_TKO_Doctor_Stoppage', 'B_Stance', 'B_Height_cms',
       'B_Reach_cms', 'B_Weight_lbs', 'R_avg_KD', 'R_avg_SIG_STR_pct',
       'R_avg_TD_pct', 'R_avg_SUB_ATT', 'R_avg_REV', 'R_avg_TD_att',
       'R_avg_TD_landed', 'R_avg_CTRL_time', 'R_total_time_fought',
       'R_total_rounds_fought', 'R_total_title_bouts', 'R_wins', 'R_losses',
       'R_win_by_Decision_Majority', 'R_win_by_Decision_Split',
       'R_win_by_Decision_Unanimous', 'R_win_by_KO/TKO', 'R_win_by_Submission',
       'R_win_by_TKO_Doctor_Stoppage', 'R_Stance', 'R_Height_cms',
       'R_Reach_cms', 'R_Weight_lbs', 'B_age', 'R_age', 'B_PCT_STRIKES',
       'R_PCT_STRIKES', 'B_PCT_HEAD', 'R_PCT_HEAD', 'B_PCT_BODY', 'R_PCT_BODY',
       'B_PCT_LEG', 'R_PCT_LEG', 'B_PCT_DISTANCE', 'R_PCT_DISTANCE',
       'B_PCT_CLINCH', 'R_PCT_CLINCH', 'B_PCT_GROUND', 'R_PCT_GROUND'}

In [ ]:
data.B_avg_KD

In [ ]:
'''plt.figure(figsize=(50, 40))

corr_matrix = data.iloc[:,:].corr(method = 'pearson').abs()

sns.heatmap(corr_matrix, annot=True)'''

In [ ]:
'''correlations = (corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool)).stack()
                    .sort_values(ascending=False))
print(correlations[:10])'''

In [ ]:
# FUNCTION FOR SELECT THE LAST OR FIRST FIGHT OF A CONCRETE FIGHTER

#  i = index of the fighter's fight, 0 means the last fight, -1 means first fight
def select_fight_row(df, name, i): 
    
    df_temp = df[(df['R_fighter'] == name) | (df['B_fighter'] == name)]  # filter df on fighter's name
    
    df_temp.reset_index(drop=True, inplace=True) #  as we created a new temporary dataframe, we have to reset indexes
    
    idx = max(df_temp.index)  #  get the index of the oldest fight
    
    if i > idx:  #  if we are looking for a fight that didn't exist, we return nothing
        
        return
    
    arr = df_temp.iloc[i].values
    
    return arr

select_fight_row(data, 'GLOVER TEIXEIRA', 0) #  we get the last fight of Jon Jones, for example

In [ ]:
# GET ALL ACTIVE FIGHTERS ACCORDING TO THE LIMIT DATE

def list_fighters(df, limit_date):
    
    df_temp = df[df['date'] > limit_date]
    
    set_R = set(df_temp['R_fighter'])
    set_B = set(df_temp['B_fighter'])
    
    fighters = list(set_R.union(set_B))
    
    return fighters

fighters = list_fighters(data, '2002-01-01')
len(fighters)

In [ ]:
# FUNCTION FOR BUILD A DATAFRAME WHICH HAS LAST/FIRST/... FIGHTS OF SELECTED FIGHTERS

def build_df(df, fighters, i):
    
    arr = [select_fight_row(df, fighters[f], i) for f in range(len(fighters)) if select_fight_row(df, fighters[f], i) is not None]
    
    cols = [col for col in df] 
    
    df_fights = pd.DataFrame(data=arr, columns=cols)
    
    df_fights.drop_duplicates(inplace=True)
    
    df_fights['title_bout'] = df_fights['title_bout'].replace({True: 1, False: 0})
    
    df_fights.drop(['R_fighter', 'B_fighter', 'date'], axis=1, inplace=True)
    
    return df_fights

df_train = build_df(data, fighters, 0)
df_test = build_df(data, fighters, 1)

In [ ]:
df_train.shape, df_test.shape

In [ ]:
# AMOUNT OF WINNERS RED OR BLUE IN TRAIN AND TEST DATAFRAMES

print(len(df_train[df_train['Winner'] == 'Blue']))
print(len(df_train[df_train['Winner'] == 'Red']))

print(len(df_test[df_test['Winner'] == 'Blue']))
print(len(df_test[df_test['Winner'] == 'Red']))

In [ ]:
# TRANSFORM DATA FOR LabelEncoder TO OBJECT COLUMNS

from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.compose import make_column_transformer

preprocessor = make_column_transformer((OrdinalEncoder(), ['weight_class', 'B_Stance', 'R_Stance']),
                                       remainder='passthrough')

# If the winner is from the Red corner, Winner label will be encoded as 1, otherwise it will be 0 (Blue corner)

label_encoder = LabelEncoder()

y_train = label_encoder.fit_transform(df_train['Winner'])
y_test = label_encoder.transform(df_test['Winner'])

X_train, X_test = df_train.drop(['Winner'], axis=1), df_test.drop(['Winner'], axis=1)

In [ ]:
print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

# Random Forest composed of 100 decision trees.
# We optimized parameters using cross-validation and GridSearch tool paired together
random_forest = RandomForestClassifier(n_estimators=100, 
                                       criterion='entropy',   
                                       random_state=42)

model = Pipeline([('encoding', preprocessor), ('random_forest', random_forest)])
model.fit(X_train, y_train)

# We use cross-validation with 5-folds to have a more precise accuracy (reduce variation)
accuracies = cross_val_score(estimator=model, X=X_train, y=y_train)
print('Accuracy mean : ', accuracies)
print('Accuracy standard deviation : ', accuracies)

y_pred = model.predict(X_test)
print('Testing accuracy : ', accuracy_score(y_test, y_pred), '\n')

target_names = ["Blue","Red"]
print(classification_report(y_test, y_pred, labels=[0,1], target_names=target_names))

In [ ]:
'''from sklearn.model_selection import GridSearchCV
parameters = [{'random_forest__n_estimators': [10, 50, 100, 500, 1000],
               'random_forest__criterion': ['gini', 'entropy'],
               'random_forest__max_depth': [5, 10, 50],
               'random_forest__min_samples_split': [2, 3, 4],
               'random_forest__min_samples_leaf': [1, 2, 3],
              }]
model = Pipeline([('encoding', preprocessor), ('random_forest', RandomForestClassifier())])

grid_search = GridSearchCV(estimator=model, param_grid=parameters, scoring='accuracy', cv=5, n_jobs=-1)
grid_search = grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_

best_params = grid_search.best_params_
print('Best accuracy : ', best_accuracy)
print('Best parameters : ', best_params)'''

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred) 
ax = plt.subplot()
sns.heatmap(cm, annot = True, ax = ax, fmt = "d")

ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')
ax.set_title("Confusion Matrix")

ax.xaxis.set_ticklabels(['Blue', 'Red'])
ax.yaxis.set_ticklabels(['Blue', 'Red'])

In [ ]:
'''feature_names = [col for col in X_train]
feature_importances = model['random_forest'].feature_importances_
indices = np.argsort(feature_importances)[::-1]
n = 30 # maximum feature importances displayed
idx = indices[0:n] 
std = np.std([tree.feature_importances_ for tree in model['random_forest'].estimators_], axis=0)

#for f in range(n):
#    print("%d. feature %s (%f)" % (f + 1, feature_names[idx[f]], feature_importances[idx[f]])) 

plt.figure(figsize=(30, 8))
plt.title("Feature importances")
plt.bar(range(n), feature_importances[idx], color="r", yerr=std[idx], align="center")
plt.xticks(range(n), [feature_names[id] for id in idx], rotation = 45) 
plt.xlim([-1, n]) 
plt.show()'''

In [ ]:
def predict(df, pipeline, blue_fighter, red_fighter, weightclass, rounds, title_bout=False): 
    
    #We build two dataframes, one for each figther 
    f1 = df[(df['R_fighter'] == blue_fighter) | (df['B_fighter'] == blue_fighter)].copy()
    f1.reset_index(drop=True, inplace=True)
    f1 = f1[:1]
    f2 = df[(df['R_fighter'] == red_fighter) | (df['B_fighter'] == red_fighter)].copy()
    f2.reset_index(drop=True, inplace=True)
    f2 = f2[:1]
    
    # if the fighter was red/blue corner on his last fight, we filter columns to only keep his statistics (and not the other fighter)
    # then we rename columns according to the color of  the corner in the parameters using re.sub()
    if (f1.loc[0, ['R_fighter']].values[0]) == blue_fighter:
        result1 = f1.filter(regex='^R', axis=1).copy() #here we keep the red corner stats
        result1.rename(columns = lambda x: re.sub('^R','B', x), inplace=True)  #we rename it with "B_" prefix because he's in the blue_corner
    else: 
        result1 = f1.filter(regex='^B', axis=1).copy()
    if (f2.loc[0, ['R_fighter']].values[0]) == red_fighter:
        result2 = f2.filter(regex='^R', axis=1).copy()
    else:
        result2 = f2.filter(regex='^B', axis=1).copy()
        result2.rename(columns = lambda x: re.sub('^B','R', x), inplace=True)
        
    fight = pd.concat([result1, result2], axis = 1) # we concatenate the red and blue fighter dataframes (in columns)
    fight.drop(['R_fighter','B_fighter'], axis = 1, inplace = True) # we remove fighter names
    fight.insert(0, 'title_bout', title_bout) # we add tittle_bout, weight class and number of rounds data to the dataframe
    fight.insert(1, 'weight_class', weightclass)
    fight.insert(2, 'no_of_rounds', rounds)
    fight['title_bout'] = fight['title_bout'].replace({True: 1, False: 0})
    
    pred = pipeline.predict(fight)
    proba = pipeline.predict_proba(fight)
    if (pred == 1.0): 
        print("The predicted winner is", red_fighter, 'with a probability of', round(proba[0][1] * 100, 2), "%")
    else:
        print("The predicted winner is", blue_fighter, 'with a probability of ', round(proba[0][0] * 100, 2), "%")
    return proba

In [ ]:
predict(data, model, 'JIRI PROCHAZKA', 'GLOVER TEIXEIRA', 'LightHeavyweight', 5, True) 

In [ ]:
predict(data, model, 'GLOVER TEIXEIRA', 'JIRI PROCHAZKA', 'LightHeavyweight', 5, True) 

In [ ]:
predict(data, model, 'VALENTINA SHEVCHENKO', 'TAILA SANTOS', 'WomenFlyweight', 5, True) 

In [ ]:
predict(data, model, 'TAILA SANTOS', 'VALENTINA SHEVCHENKO', 'WomenFlyweight', 5, True) 

In [ ]:
predict(data, model, 'ISRAEL ADESANYA', 'JARED CANNONIER', 'Middleweight', 5, True) 

In [ ]:
predict(data, model, 'JARED CANNONIER', 'ISRAEL ADESANYA', 'Middleweight', 5, True) 

In [ ]:
predict(data, model, 'ALEXANDER VOLKANOVSKI', 'MAX HOLLOWAY', 'Featherweight', 5, True) 

In [ ]:
predict(data, model, 'MAX HOLLOWAY', 'ALEXANDER VOLKANOVSKI', 'Featherweight', 5, True) 

In [ ]:
'''# buscando colinealidad

plt.figure(figsize=(15, 10))

sns.set(style='white')

mask=np.triu(np.ones_like(data.corr(), dtype=bool))

cmap=sns.diverging_palette(0, 10, as_cmap=True)


sns.heatmap(data.corr(),
           mask=mask,
          cmap=cmap,
          center=0,
          square=True,
          linewidths=0.5,
          cbar_kws={'shrink': 0.5},
           annot=True);'''

### LOGISTIC REGRESSION AND RANDOM FOREST

In [ ]:
'''# label encoding 

le=LabelEncoder()

for c in data.columns:
    
    if data.dtypes[c]==object:
        
        le.fit(data[c].astype(str))
        
        data[c]=le.transform(data[c].astype(str))

X = data.drop('Winner', axis=1)

y = data.Winner

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, train_size=0.8,
                                                    random_state=42)

logreg=LogisticRegression(max_iter=2000)

# RMSE POR MODELO
models = [logreg, rfr]

for m in models:
    
    m.fit(X_train, y_train)
    y_pred = m.predict(X_test)
    
    print(f'RMSE Model {m}: {mse(y_test, y_pred, squared=False)}')
    print(f'R2 Model {m}: {r2(y_test, y_pred)}')
    
sum(y_pred==y_test)/y_pred.shape[0]*100'''

### H2O

In [ ]:
'''train = data.iloc[:4580]

test = data.iloc[4580:].drop('Winner', axis=1)

train.shape, test.shape'''

In [ ]:
'''# H2O

h2o.init()

# parsear datos para h20

h2train=h2o.H2OFrame(train)

h2test=h2o.H2OFrame(test)

X=[c for c in h2train.columns if c!='Winner']

y='Winner'

# inicia auto-machine-learning

automl=H2OAutoML(max_models=50,
                 seed=42,
                 max_runtime_secs=300,
                 sort_metric='RMSE')'''

In [ ]:
'''# entrena

automl.train(x=X,
             y=y,
             training_frame=h2train)'''

In [ ]:
'''# prediciones del lider

y_pred = automl.leader.predict(h2test)

y_pred = y_pred.as_data_frame()

y_pred'''